In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdriv')

Drive already mounted at /content/gdriv; to attempt to forcibly remount, call drive.mount("/content/gdriv", force_remount=True).


In [ ]:
import os
data_path = "/content/gdriv/My Drive/Colab Notebooks/Data" 
dt1 = pd.read_csv(data_path + '/fine_dust.csv')
dt2 = pd.read_csv(data_path + '/fine_dust_without_y.csv')

In [ ]:
# dt라는 이름으로 concat
dt = pd.concat([dt1, dt2],ignore_index=True)
dt

,PM,PM2,EG1,EG2,EG3,EG4,EG5,EG6,EG7,EG8,EG9,EG10,EG11,EG12,EG13,EG14,EG15,EG16,EG17,EG18,EG19,EG20,HUM,TMP,WD,WS
0,27.0,25.0,1305.26,15069.82,12815.28,2847.84,2847.84,1230.46,14206.22,12080.88,2684.64,2684.64,291.72,3368.04,2864.16,636.48,636.48,583.44,6736.08,5728.32,1272.96,1272.96,91,7.2,0.906308,0.6
1,52.0,17.0,1021.02,11788.14,10024.56,2227.68,2227.68,729.30,8420.10,7160.40,1591.20,1591.20,0.00,0.00,0.00,0.00,0.00,437.58,5052.06,4296.24,954.72,954.72,36,14.3,0.438371,1.3
2,22.0,12.0,867.68,10017.76,8519.04,1893.12,1893.12,1301.52,15026.64,12778.56,2839.68,2839.68,145.86,1684.02,1432.08,318.24,318.24,291.72,3368.04,2864.16,636.48,636.48,38,6.7,0.992546,0.5
3,23.0,13.0,729.30,8420.10,7160.40,1591.20,1591.20,729.30,8420.10,7160.40,1591.20,1591.20,0.00,0.00,0.00,0.00,0.00,145.86,1684.02,1432.08,318.24,318.24,23,9.7,0.241922,3.0
4,90.0,63.0,946.22,10924.54,9290.16,2064.48,2064.48,575.96,6649.72,5654.88,1256.64,1256.64,0.00,0.00,0.00,0.00,0.00,583.44,6736.08,5728.32,1272.96,1272.96,18,5.9,0.484810,1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3523,NaN,NaN,1237.94,14292.58,12154.32,2700.96,2700.96,1806.42,20855.94,17735.76,3941.28,3941.28,291.72,3368.04,2864.16,636.48,636.48,1155.66,13342.62,11346.48,2521.44,2521.44,68,13.1,-0.997564,0.3
3524,NaN,NaN,875.16,10104.12,8592.48,1909.44,1909.44,583.44,6736.08,5728.32,1272.96,1272.96,291.72,3368.04,2864.16,636.48,636.48,437.58,5052.06,4296.24,954.72,954.72,58,5.9,0.544639,2.0
3525,NaN,NaN,1021.02,11788.14,10024.56,2227.68,2227.68,437.58,5052.06,4296.24,954.72,954.72,721.82,8333.74,7086.96,1574.88,1574.88,0.00,0.00,0.00,0.00,0.00,48,8.4,0.515038,1.3
3526,NaN,NaN,583.44,6736.08,5728.32,1272.96,1272.96,729.30,8420.10,7160.40,1591.20,1591.20,1006.06,11615.42,9877.68,2195.04,2195.04,654.50,7556.50,6426.00,1428.00,1428.00,51,2.8,0.374607,0.6


In [ ]:
x = dt.iloc[:,2:]      # dt에서 PM, PM2 제외하고 추출


from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(x)
x = min_max_scaler.transform(x)

dt.iloc[:,2:] = x
dt

,PM,PM2,EG1,EG2,EG3,EG4,EG5,EG6,EG7,EG8,EG9,EG10,EG11,EG12,EG13,EG14,EG15,EG16,EG17,EG18,EG19,EG20,HUM,TMP,WD,WS
0,27.0,25.0,0.645102,0.645102,0.645102,0.645102,0.645102,0.629063,0.629063,0.629063,0.629063,0.629063,0.226087,0.226087,0.226087,0.226087,0.226087,0.445714,0.445714,0.445714,0.445714,0.445714,0.919192,0.446991,0.953150,0.109091
1,52.0,17.0,0.504621,0.504621,0.504621,0.504621,0.504621,0.372849,0.372849,0.372849,0.372849,0.372849,0.000000,0.000000,0.000000,0.000000,0.000000,0.334286,0.334286,0.334286,0.334286,0.334286,0.363636,0.650430,0.719164,0.236364
2,22.0,12.0,0.428835,0.428835,0.428835,0.428835,0.428835,0.665392,0.665392,0.665392,0.665392,0.665392,0.113043,0.113043,0.113043,0.113043,0.113043,0.222857,0.222857,0.222857,0.222857,0.222857,0.383838,0.432665,0.996273,0.090909
3,23.0,13.0,0.360444,0.360444,0.360444,0.360444,0.360444,0.372849,0.372849,0.372849,0.372849,0.372849,0.000000,0.000000,0.000000,0.000000,0.000000,0.111429,0.111429,0.111429,0.111429,0.111429,0.232323,0.518625,0.620932,0.545455
4,90.0,63.0,0.467652,0.467652,0.467652,0.467652,0.467652,0.294455,0.294455,0.294455,0.294455,0.294455,0.000000,0.000000,0.000000,0.000000,0.000000,0.445714,0.445714,0.445714,0.445714,0.445714,0.181818,0.409742,0.742385,0.236364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3523,NaN,NaN,0.611830,0.611830,0.611830,0.611830,0.611830,0.923518,0.923518,0.923518,0.923518,0.923518,0.226087,0.226087,0.226087,0.226087,0.226087,0.882857,0.882857,0.882857,0.882857,0.882857,0.686869,0.616046,0.001142,0.054545
3524,NaN,NaN,0.432532,0.432532,0.432532,0.432532,0.432532,0.298279,0.298279,0.298279,0.298279,0.298279,0.226087,0.226087,0.226087,0.226087,0.226087,0.334286,0.334286,0.334286,0.334286,0.334286,0.585859,0.409742,0.772302,0.363636
3525,NaN,NaN,0.504621,0.504621,0.504621,0.504621,0.504621,0.223709,0.223709,0.223709,0.223709,0.223709,0.559420,0.559420,0.559420,0.559420,0.559420,0.000000,0.000000,0.000000,0.000000,0.000000,0.484848,0.481375,0.757501,0.236364
3526,NaN,NaN,0.288355,0.288355,0.288355,0.288355,0.288355,0.372849,0.372849,0.372849,0.372849,0.372849,0.779710,0.779710,0.779710,0.779710,0.779710,0.500000,0.500000,0.500000,0.500000,0.500000,0.515152,0.320917,0.687279,0.109091


In [ ]:
train = dt.iloc[:3428,:]    # 3428행까지 train
test = dt.iloc[3428:,:]     # 그 이후는 test

In [ ]:
# PM 범주화

PM1_0 = train.loc[dt['PM']<=30,:].index
PM1_1 = train.loc[(dt['PM']>=31) & (dt['PM']<=80),:].index
pm1_2 = train.loc[(dt['PM']>=81),:].index

train.iloc[PM1_0,0] = 0
train.iloc[PM1_1,0] = 1
train.iloc[pm1_2,0] = 2

In [ ]:
# PM2 범주화

PM2_0 = train.loc[dt['PM2']<=15,:].index
PM2_1 = train.loc[(dt['PM2']>=15) & (dt['PM2']<=35),:].index
pm2_2 = train.loc[(dt['PM2']>=36),:].index

train.iloc[PM2_0,1] = 0
train.iloc[PM2_1,1] = 1
train.iloc[pm2_2,1] = 2

In [ ]:
# train PM, PM2 결과

train

,PM,PM2,EG1,EG2,EG3,EG4,EG5,EG6,EG7,EG8,EG9,EG10,EG11,EG12,EG13,EG14,EG15,EG16,EG17,EG18,EG19,EG20,HUM,TMP,WD,WS
0,0.0,1.0,0.645102,0.645102,0.645102,0.645102,0.645102,0.629063,0.629063,0.629063,0.629063,0.629063,0.226087,0.226087,0.226087,0.226087,0.226087,0.445714,0.445714,0.445714,0.445714,0.445714,0.919192,0.446991,0.953150,0.109091
1,1.0,1.0,0.504621,0.504621,0.504621,0.504621,0.504621,0.372849,0.372849,0.372849,0.372849,0.372849,0.000000,0.000000,0.000000,0.000000,0.000000,0.334286,0.334286,0.334286,0.334286,0.334286,0.363636,0.650430,0.719164,0.236364
2,0.0,0.0,0.428835,0.428835,0.428835,0.428835,0.428835,0.665392,0.665392,0.665392,0.665392,0.665392,0.113043,0.113043,0.113043,0.113043,0.113043,0.222857,0.222857,0.222857,0.222857,0.222857,0.383838,0.432665,0.996273,0.090909
3,0.0,0.0,0.360444,0.360444,0.360444,0.360444,0.360444,0.372849,0.372849,0.372849,0.372849,0.372849,0.000000,0.000000,0.000000,0.000000,0.000000,0.111429,0.111429,0.111429,0.111429,0.111429,0.232323,0.518625,0.620932,0.545455
4,2.0,2.0,0.467652,0.467652,0.467652,0.467652,0.467652,0.294455,0.294455,0.294455,0.294455,0.294455,0.000000,0.000000,0.000000,0.000000,0.000000,0.445714,0.445714,0.445714,0.445714,0.445714,0.181818,0.409742,0.742385,0.236364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423,1.0,1.0,0.752311,0.752311,0.752311,0.752311,0.752311,0.369025,0.369025,0.369025,0.369025,0.369025,0.675362,0.675362,0.675362,0.675362,0.675362,0.445714,0.445714,0.445714,0.445714,0.445714,0.383838,0.283668,0.320764,0.181818
3424,1.0,1.0,0.360444,0.360444,0.360444,0.360444,0.360444,0.298279,0.298279,0.298279,0.298279,0.298279,0.446377,0.446377,0.446377,0.446377,0.446377,0.222857,0.222857,0.222857,0.222857,0.222857,0.545455,0.518625,0.280760,0.163636
3425,0.0,0.0,0.500924,0.500924,0.500924,0.500924,0.500924,0.223709,0.223709,0.223709,0.223709,0.223709,0.000000,0.000000,0.000000,0.000000,0.000000,0.557143,0.557143,0.557143,0.557143,0.557143,0.555556,0.830946,0.320764,0.236364
3426,1.0,2.0,0.432532,0.432532,0.432532,0.432532,0.432532,0.298279,0.298279,0.298279,0.298279,0.298279,0.000000,0.000000,0.000000,0.000000,0.000000,0.665714,0.665714,0.665714,0.665714,0.665714,0.333333,0.083095,0.637791,0.454545


In [ ]:
x = train.iloc[:,2:]    

pm1 = train.iloc[:,0]    # train에서 PM 추출
pm2 = train.iloc[:,1]    # train에서 PM2 추출

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class mlp(nn.Module):
    def __init__(self, input_size):
        super(mlp, self).__init__()
        self.input_size = input_size
        self.fc1 = nn.Linear(self.input_size,18)
        self.bn1 = torch.nn.BatchNorm1d(18)
        self.fc2 = nn.Linear(18,9)
        self.bn2 = torch.nn.BatchNorm1d(9)
        self.fc4 = nn.Linear(9,3)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout()
        self.sf = torch.nn.Softmax()
    
    def forward(self, x):
        output = self.fc1(x)
        output = self.bn1(output)
        output = self.relu(output)
        output = self.dropout(output)
        output = self.fc2(output)
        output = self.bn2(output)
        output = self.relu(output)
        output = self.dropout(output)
        output = self.fc4(output)
        output = self.sf(output)
        return output

In [ ]:
x = x.to_numpy()
pm1 = pm1.to_numpy()
pm2 = pm2.to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

pm1_trainx, pm1_testx, pm1_trainy, pm1_testy = train_test_split(x, pm1, test_size=0.3, random_state = 15)
pm2_trainx, pm2_testx, pm2_trainy, pm2_testy = train_test_split(x, pm2, test_size=0.3, random_state = 15)

In [ ]:
pm1_trainx = torch.FloatTensor(pm1_trainx)
pm1_testx = torch.FloatTensor(pm1_testx)

pm2_trainx = torch.FloatTensor(pm2_trainx)
pm2_testx = torch.FloatTensor(pm2_testx)

In [ ]:
pm1_trainy = torch.FloatTensor(pm1_trainy).view(-1)
pm1_trainy = pm1_trainy.long()

pm1_testy = torch.FloatTensor(pm1_testy).view(-1)
pm1_testy = pm1_testy.long()


pm2_trainy = torch.FloatTensor(pm2_trainy).view(-1)
pm2_trainy = pm2_trainy.long()

pm2_testy = torch.FloatTensor(pm2_testy).view(-1)
pm2_testy = pm2_testy.long()

In [ ]:
input_size = pm1_trainx.size()[1]
model1 = mlp(input_size)

In [ ]:
input_size = pm2_trainx.size()[1]
model2 = mlp(input_size)

In [ ]:
optimizer1 = optim.Adam(model1.parameters(), lr = 0.01)
loss = nn.CrossEntropyLoss()

In [ ]:
optimizer2 = optim.Adam(model2.parameters(), lr = 0.01)
loss = nn.CrossEntropyLoss()

In [ ]:
# PM 체크하는 과정

for e in range(5000):
  optimizer1.zero_grad()
  out = model1(pm1_trainx)
  L = loss(out,pm1_trainy)
  L.backward()
  optimizer1.step()
  if (e+1) % 500 == 0:
    with torch.no_grad():
      model1.eval()
      pred = model1(pm1_testx)
      acc = sum(torch.max(pred,1)[1] == pm1_testy).numpy()/len(pm1_testy)

    print('[Epoch: {}/{}] [Loss: {:2f}] [Test Accuracy: {:2f}]'.format(e+1, 5000, L.item(), acc*100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch: 500/5000] [Loss: 0.880137] [Test Accuracy: 66.666667]
[Epoch: 1000/5000] [Loss: 0.793880] [Test Accuracy: 67.638484]
[Epoch: 1500/5000] [Loss: 0.781155] [Test Accuracy: 67.249757]
[Epoch: 2000/5000] [Loss: 0.772781] [Test Accuracy: 68.027211]
[Epoch: 2500/5000] [Loss: 0.828668] [Test Accuracy: 67.346939]
[Epoch: 3000/5000] [Loss: 0.783921] [Test Accuracy: 68.610301]
[Epoch: 3500/5000] [Loss: 0.777377] [Test Accuracy: 68.707483]
[Epoch: 4000/5000] [Loss: 0.785872] [Test Accuracy: 68.901846]
[Epoch: 4500/5000] [Loss: 0.771411] [Test Accuracy: 68.124393]
[Epoch: 5000/5000] [Loss: 0.775267] [Test Accuracy: 68.901846]


In [ ]:
# PM2 체크하는 과정

for e in range(5000):
  optimizer2.zero_grad()
  out = model2(pm2_trainx)
  L = loss2(out,pm2_trainy)
  L.backward()
  optimizer2.step()
  if (e+1) % 500 == 0:
    with torch.no_grad():
      model2.eval()
      pred = model2(pm2_testx)
      acc = sum(torch.max(pred,1)[1] == pm2_testy).numpy()/len(pm2_testy)

    print('[Epoch: {}/{}] [Loss: {:2f}] [Test Accuracy: {:2f}]'.format(e+1, 5000, L.item(), acc*100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch: 500/5000] [Loss: 0.962173] [Test Accuracy: 60.447036]
[Epoch: 1000/5000] [Loss: 0.852808] [Test Accuracy: 63.265306]
[Epoch: 1500/5000] [Loss: 0.837854] [Test Accuracy: 62.682216]
[Epoch: 2000/5000] [Loss: 0.823278] [Test Accuracy: 63.556851]
[Epoch: 2500/5000] [Loss: 0.833247] [Test Accuracy: 64.625850]
[Epoch: 3000/5000] [Loss: 0.824207] [Test Accuracy: 63.362488]
[Epoch: 3500/5000] [Loss: 0.837962] [Test Accuracy: 63.459670]
[Epoch: 4000/5000] [Loss: 0.820844] [Test Accuracy: 63.168124]
[Epoch: 4500/5000] [Loss: 0.828739] [Test Accuracy: 62.682216]
[Epoch: 5000/5000] [Loss: 0.838693] [Test Accuracy: 62.682216]


In [ ]:
test_x = test.iloc[:,2:]
test_x = test_x.to_numpy()

add_pm = torch.FloatTensor(test_x)
add_pm2 = torch.FloatTensor(test_x)

In [ ]:
add_pm1 = model1(add_pm)
add_pm1 = torch.max(add_pm1,1)[1]
dt2['PM'] = add_pm1
dt2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


,EG1,EG2,EG3,EG4,EG5,EG6,EG7,EG8,EG9,EG10,EG11,EG12,EG13,EG14,EG15,EG16,EG17,EG18,EG19,EG20,HUM,TMP,WD,WS,PM
0,583.44,6736.08,5728.32,1272.96,1272.96,583.44,6736.08,5728.32,1272.96,1272.96,437.58,5052.06,4296.24,954.72,954.72,437.58,5052.06,4296.24,954.72,954.72,68,23.7,0.882948,0.7,0
1,875.16,10104.12,8592.48,1909.44,1909.44,1151.92,13299.44,11309.76,2513.28,2513.28,437.58,5052.06,4296.24,954.72,954.72,583.44,6736.08,5728.32,1272.96,1272.96,34,3.5,0.190809,3.5,1
2,729.30,8420.10,7160.40,1591.20,1591.20,1092.08,12608.56,10722.24,2382.72,2382.72,0.00,0.00,0.00,0.00,0.00,437.58,5052.06,4296.24,954.72,954.72,58,11.9,0.754710,0.6,1
3,875.16,10104.12,8592.48,1909.44,1909.44,1600.72,18481.04,15716.16,3492.48,3492.48,437.58,5052.06,4296.24,954.72,954.72,291.72,3368.04,2864.16,636.48,636.48,25,7.0,0.275637,1.9,0
4,1237.94,14292.58,12154.32,2700.96,2700.96,1806.42,20855.94,17735.76,3941.28,3941.28,291.72,3368.04,2864.16,636.48,636.48,1301.52,15026.64,12778.56,2839.68,2839.68,66,13.7,-0.173648,0.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1237.94,14292.58,12154.32,2700.96,2700.96,1806.42,20855.94,17735.76,3941.28,3941.28,291.72,3368.04,2864.16,636.48,636.48,1155.66,13342.62,11346.48,2521.44,2521.44,68,13.1,-0.997564,0.3,1
96,875.16,10104.12,8592.48,1909.44,1909.44,583.44,6736.08,5728.32,1272.96,1272.96,291.72,3368.04,2864.16,636.48,636.48,437.58,5052.06,4296.24,954.72,954.72,58,5.9,0.544639,2.0,1
97,1021.02,11788.14,10024.56,2227.68,2227.68,437.58,5052.06,4296.24,954.72,954.72,721.82,8333.74,7086.96,1574.88,1574.88,0.00,0.00,0.00,0.00,0.00,48,8.4,0.515038,1.3,1
98,583.44,6736.08,5728.32,1272.96,1272.96,729.30,8420.10,7160.40,1591.20,1591.20,1006.06,11615.42,9877.68,2195.04,2195.04,654.50,7556.50,6426.00,1428.00,1428.00,51,2.8,0.374607,0.6,1


In [ ]:
addd_pm2 = model2(add_pm2)
addd_pm2 = torch.max(addd_pm2,1)[1]
dt2['PM2'] = addd_pm2
dt2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


,EG1,EG2,EG3,EG4,EG5,EG6,EG7,EG8,EG9,EG10,EG11,EG12,EG13,EG14,EG15,EG16,EG17,EG18,EG19,EG20,HUM,TMP,WD,WS,PM,PM2
0,583.44,6736.08,5728.32,1272.96,1272.96,583.44,6736.08,5728.32,1272.96,1272.96,437.58,5052.06,4296.24,954.72,954.72,437.58,5052.06,4296.24,954.72,954.72,68,23.7,0.882948,0.7,0,0
1,875.16,10104.12,8592.48,1909.44,1909.44,1151.92,13299.44,11309.76,2513.28,2513.28,437.58,5052.06,4296.24,954.72,954.72,583.44,6736.08,5728.32,1272.96,1272.96,34,3.5,0.190809,3.5,1,1
2,729.30,8420.10,7160.40,1591.20,1591.20,1092.08,12608.56,10722.24,2382.72,2382.72,0.00,0.00,0.00,0.00,0.00,437.58,5052.06,4296.24,954.72,954.72,58,11.9,0.754710,0.6,1,1
3,875.16,10104.12,8592.48,1909.44,1909.44,1600.72,18481.04,15716.16,3492.48,3492.48,437.58,5052.06,4296.24,954.72,954.72,291.72,3368.04,2864.16,636.48,636.48,25,7.0,0.275637,1.9,0,2
4,1237.94,14292.58,12154.32,2700.96,2700.96,1806.42,20855.94,17735.76,3941.28,3941.28,291.72,3368.04,2864.16,636.48,636.48,1301.52,15026.64,12778.56,2839.68,2839.68,66,13.7,-0.173648,0.4,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1237.94,14292.58,12154.32,2700.96,2700.96,1806.42,20855.94,17735.76,3941.28,3941.28,291.72,3368.04,2864.16,636.48,636.48,1155.66,13342.62,11346.48,2521.44,2521.44,68,13.1,-0.997564,0.3,1,2
96,875.16,10104.12,8592.48,1909.44,1909.44,583.44,6736.08,5728.32,1272.96,1272.96,291.72,3368.04,2864.16,636.48,636.48,437.58,5052.06,4296.24,954.72,954.72,58,5.9,0.544639,2.0,1,1
97,1021.02,11788.14,10024.56,2227.68,2227.68,437.58,5052.06,4296.24,954.72,954.72,721.82,8333.74,7086.96,1574.88,1574.88,0.00,0.00,0.00,0.00,0.00,48,8.4,0.515038,1.3,1,1
98,583.44,6736.08,5728.32,1272.96,1272.96,729.30,8420.10,7160.40,1591.20,1591.20,1006.06,11615.42,9877.68,2195.04,2195.04,654.50,7556.50,6426.00,1428.00,1428.00,51,2.8,0.374607,0.6,1,1


In [ ]:
dt2.to_csv('PM,PM2 삽입결과.csv', index=False)